# Sitzung 4

Diese Skripte sind ausschließlich als Zusatz-Material gedacht. Speziell für diejenigen unter Euch, die einen Einblick in das Programmieren gewinnen wollen. Wenn Du es also leid bist repetitive Tätigkeiten auszuführen und das lieber einer Maschine überlassen willst, bist Du hier genau richtig. 

<span style="color:red"> Die Codes sind nicht für die Klausur relevant, genau genommen haben sie mit dem Lehrstuhl für Statistik __rein gar nichts__ zu tun. </span>

Dieses mal sehen wir uns wieder Verteilungen an, der Star diesmal: die __Hypergeometrische Verteilung__. Sie ist eng mit der Binomialverteilung verwandt und ist angemessen, wenn sich die Wahrscheinlichkeit zwischen den einzelnen Experimenten verändert.

Wir werden wieder versuchen eine Intuition für die Verteilung zu gewinnen, indem wir uns interaktive plots ansehen.

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider

Wir implementieren wieder einmal die mathematische Wahrscheinlichkeitsfunktion

$$
f_H (x;N;M;n) = \begin{cases} 
      \frac{{M \choose x} {N-M \choose n-x}}{{N \choose n}} \quad  \text{wenn x} \in \left\{ 0, 1, \dots, \min(n, M)\right\}\\
      0 \quad sonst
   \end{cases}
$$

In [3]:
def bincoef(n, k):
    return np.math.factorial(n) / (np.math.factorial(k) * np.math.factorial(n-k))


def pbinom(x, p, n):
    """Berechne die Wahrscheinlichkeit, dass X=x, für X ~ Binom(n, p)"""
    binom_coeff = (np.math.factorial(n)) / (np.math.factorial(x) * np.math.factorial(n - x))
    return binom_coeff * p**x * (1-p)**(n-x)


def phyper(x, N, M, n):
    """Berechne die Wahrscheinlichkeit, dass X=x, für X ~ Hyper(N, M, n)"""
    # Teste ob die Parameter sinnvoll gewählt wurden.
    assert M <= N
    assert x <= n
    
    try:
        num_positive = bincoef(M, x)
        num_negative = bincoef(N-M, n-x)
        total = bincoef(N, n)
    except ValueError:
        return 0
    
    return (num_positive * num_negative) / total

Ganz analog zu letztem Mal, definieren wir eine Funktion, die die plots generiert.

In [4]:
def view_pdf(N, M, n, binom):
    """Plotte Verteilung von X~Hyper(N, M, n) und Y~Bin(n, M/N)."""
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(11, 11))    
    
    if binom:
        w = 0.5
        xmin = max(0, n - (N-M))
        xmax = min(n, M)

        if xmin:
            xmin_label = r"$X_{min} = {%d} - ({%d} - {%d}) = {%d}$" % (n, N, M, xmin)
        else:
            xmin_label = r"$X_{min}:$ 0"
        
        xmax_label = r"$X_{max} = \min({%d}, {%d}) = {%d}$" % (n, M, xmax)        
        
        X_ = np.arange(xmin, xmax + 1, 1)
        y_hyper = [phyper(x, N=N, M=M, n=n) for x in X_]
        y_binom = [pbinom(x, n=n, p=M/N) for x in X_]
        
        ax.plot([], [], " ", label=xmin_label)
        ax.plot([], [], " ", label=xmax_label)
        ax.bar(X_ - w/2, y_hyper, width=w, label="X ~ Hyper({}, {}, {})".format(N, M, n))
        ax.bar(X_ + w/2, y_binom, width=w, label="Y ~ Bin({}, {:3.2})".format(n, M/N))
        
    else:
        xmin = max(0, n - (N-M))
        xmax = min(n, M)

        if xmin:
            xmin_label = r"$X_{min} = {%d} - ({%d} - {%d}) = {%d}$" % (n, N, M, xmin)
        else:
            xmin_label = r"$X_{min}:$ 0"
        
        xmax_label = r"$X_{max} = \min({%d}, {%d}) = {%d}$" % (n, M, xmax)
    
        X_ = np.arange(xmin, xmax + 1, 1)
        y_hyper = [phyper(x, N=N, M=M, n=n) for x in X_]
        
        ax.bar(X_, y_hyper, label="X ~ Hyper({}, {}, {})".format(N, M, n))
        ax.plot([], [], " ", label=xmin_label)
        ax.plot([], [], " ", label=xmax_label)

        
    # Labels get too crowded, when the support becomes too large.
    if len(X_) <= 30:
        plt.xticks(X_)
    
    plt.title("Die Hypergeometrische Verteilung")
    plt.ylabel(r"$f(x)$")
    plt.xlabel(r"$X=x$")
    plt.legend()
    plt.show()


Anders ist diesmal wie wir die UI definieren. Würden wir die UI genauso wie letztes mal generieren, wären unsinnige Kombinationen möglich. Bei der hypergeometrischen Verteilung hängen $n$ und $M$ von $N$ ab. 

* $n \leq N$: weil wir nicht mehr ziehen können ($n$), als da ist ($N$)

* $M \leq N$: weil die maximale Anzahl Treffer ($M$) nicht größer sein kann, als die Gesamtanzahl ($N$)

Im besten Fall, werden unsinnige Werte in unserer Funktion `phyper` abgefangen. Im schlechtesten Fall haben wir etwas übersehen, was später zu unvorhergesehenem Verhalten führt (ein sehr häufiges Problem beim Programmieren).

In [5]:
N_widget = IntSlider(min=0, max=200, value=10, description="N:")
M_widget = IntSlider(min=0, max=N_widget.value, value=5, description="M:")
n_widget = IntSlider(min=1, max=N_widget.value, value=5, description="n:")


def updater(*args):
    M_widget.max = N_widget.value
    n_widget.max = N_widget.value
    
N_widget.observe(updater, 'value')

interact(view_pdf, N=N_widget, M=M_widget, n=n_widget, binom=False);

interactive(children=(IntSlider(value=10, description='N:', max=200), IntSlider(value=5, description='M:', max…